# Carbon instensity of Sustainable PV for Energy Transition
This analysis explores the carbon implications of different PV sustainability/circular economy designs in the context of achieving energy transition. These calculations build upon previous work that can be found in journals 13 and 17.

Attempt 1

In [1]:
import numpy as np
import pandas as pd
import os,sys
from pathlib import Path
import matplotlib.pyplot as plt

cwd = os.getcwd() #grabs current working directory

testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP' / 'CarbonAnalysis')
inputfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP')
baselinesfolder = str(Path().resolve().parent.parent /'PV_ICE' / 'baselines')
supportMatfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'baselines' / 'SupportingMaterial')
carbonfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE'/ 'baselines'/ 'CarbonLayer')
#altBaselinesfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'baselines' / 'Energy_CellModuleTechCompare')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)

In [2]:
from platform import python_version 
print(python_version())

3.8.8


In [3]:
import PV_ICE
PV_ICE.__version__

'v0.2.0+512.g0b57d08.dirty'

In [4]:
MATERIALS = ['glass', 'silicon', 'silver', 'aluminium_frames', 'copper', 'encapsulant', 'backsheet']
moduleFile_m = os.path.join(baselinesfolder, 'baseline_modules_mass_US.csv')
moduleFile_e = os.path.join(baselinesfolder, 'baseline_modules_energy.csv')

In [5]:
#load in a baseline and materials for modification
sim1 = PV_ICE.Simulation(name='sim1', path=testfolder)

sim1.createScenario(name='PV_ICE', massmodulefile=moduleFile_m, energymodulefile=moduleFile_e)
for mat in range (0, len(MATERIALS)):
    matbaseline_m = os.path.join(baselinesfolder,'baseline_material_mass_'+MATERIALS[mat]+'.csv')
    matbaseline_e = os.path.join(baselinesfolder,'baseline_material_energy_'+MATERIALS[mat]+'.csv')
    sim1.scenario['PV_ICE'].addMaterial(MATERIALS[mat], massmatfile=matbaseline_m, energymatfile=matbaseline_e)

path = C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\TEMP\CarbonAnalysis
Baseline folder directed to default:  C:\Users\hmirletz\Documents\GitHub\PV_ICE\PV_ICE\baselines


In [6]:
sim1.calculateFlows()

>>>> Calculating Material Flows <<<<

Working on Scenario:  PV_ICE
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  silicon
==> Working on Material :  silver
==> Working on Material :  aluminium_frames
==> Working on Material :  copper
==> Working on Material :  encapsulant
==> Working on Material :  backsheet


>>>> Calculating Energy Flows <<<<

Working on Scenario:  PV_ICE
********************
==> Working on Energy for Material :  glass
==> Working on Energy for Material :  silicon
==> Working on Energy for Material :  silver
==> Working on Energy for Material :  aluminium_frames
==> Working on Energy for Material :  copper
==> Working on Energy for Material :  encapsulant
==> Working on Energy for Material :  backsheet


In [7]:
sim1.calculateCarbonFlows()



>>>> Calculating Carbon Flows <<<<

Working on Scenario:  PV_ICE
********************
Working on Carbon for Module
==> Working on Carbon for Material :  glass
==> Working on Carbon for Material :  silicon
==> Working on Carbon for Material :  silver
==> Working on Carbon for Material :  aluminium_frames
==> Working on Carbon for Material :  copper
==> Working on Carbon for Material :  encapsulant
==> Working on Carbon for Material :  backsheet


In [8]:
sim1.scenario['PV_ICE'].material['encapsulant'].matdataOut_c

,Austria_vmfg_elec_gCO2eq,Brazil_vmfg_elec_gCO2eq,China_vmfg_elec_gCO2eq,France_vmfg_elec_gCO2eq,Germany_vmfg_elec_gCO2eq,India_vmfg_elec_gCO2eq,Italy_vmfg_elec_gCO2eq,Japan_vmfg_elec_gCO2eq,SKorea_vmfg_elec_gCO2eq,Netherlands_vmfg_elec_gCO2eq,...,mat_vMFG_p_gCO2eq,mat_LQmfg_p_gCO2eq,mat_LQeol_p_gCO2eq,mat_LQ_p_gCO2eq,mat_HQmfg_p_gCO2eq,mat_HQeol_p_gCO2eq,mat_HQ_p_gCO2eq,mat_vMFG_energy_gCO2eq,mat_vMFG_total_gCO2eq,mat_Recycle_e_p_gCO2eq
0,1.787519e+05,7.791768e+04,2.732055e+07,1.471878e+05,2.701880e+06,7.017471e+06,1.371764e+06,1.788924e+06,2.462092e+06,5.069888e+05,...,4.746600e+08,0.0,0.0,0.0,0.0,0.0,0.0,8.195087e+07,5.566109e+08,0.0
1,1.359721e+05,5.927002e+04,2.078205e+07,1.119621e+05,2.055252e+06,5.338014e+06,1.043467e+06,1.360790e+06,1.872852e+06,3.856536e+05,...,3.610620e+08,0.0,0.0,0.0,0.0,0.0,0.0,6.233797e+07,4.233999e+08,0.0
2,1.610565e+05,7.020426e+04,2.461596e+07,1.326171e+05,2.434409e+06,6.322781e+06,1.235967e+06,1.611831e+06,2.218359e+06,4.567997e+05,...,4.276713e+08,0.0,0.0,0.0,0.0,0.0,0.0,7.383819e+07,5.015095e+08,0.0
3,1.608918e+05,7.013245e+04,2.459079e+07,1.324814e+05,2.431919e+06,6.316313e+06,1.234703e+06,1.610182e+06,2.216090e+06,4.563325e+05,...,4.272339e+08,0.0,0.0,0.0,0.0,0.0,0.0,7.376267e+07,5.009966e+08,0.0
4,2.272904e+05,9.907553e+04,3.473920e+07,1.871554e+05,3.435551e+06,8.923002e+06,1.744254e+06,2.274691e+06,3.130651e+06,6.446571e+05,...,6.035497e+08,0.0,0.0,0.0,0.0,0.0,0.0,1.042039e+08,7.077536e+08,0.0
5,2.777194e+05,1.210574e+05,4.244679e+07,2.286796e+05,4.197797e+06,1.090275e+07,2.131253e+06,2.779377e+06,3.825249e+06,7.876873e+05,...,7.374594e+08,0.0,0.0,0.0,0.0,0.0,0.0,1.273236e+08,8.647831e+08,0.0
6,4.307294e+05,1.954007e+05,5.751332e+07,2.829479e+05,5.791578e+06,1.531302e+07,2.925682e+06,3.864668e+06,5.452151e+06,1.096479e+06,...,9.724321e+08,0.0,0.0,0.0,0.0,0.0,0.0,1.745845e+08,1.147017e+09,0.0
7,6.765990e+05,2.833725e+05,9.239641e+07,4.863174e+05,9.237312e+06,2.452297e+07,4.870488e+06,6.485668e+06,8.639870e+06,1.731665e+06,...,1.461755e+09,0.0,0.0,0.0,0.0,0.0,0.0,2.753913e+08,1.737146e+09,0.0
8,1.236527e+06,3.872835e+05,1.388350e+08,7.608554e+05,1.366093e+07,3.609426e+07,7.125236e+06,1.009551e+07,1.224517e+07,2.551002e+06,...,2.036405e+09,0.0,0.0,0.0,0.0,0.0,0.0,4.049009e+08,2.441306e+09,0.0
9,1.893417e+06,7.097430e+05,2.266740e+08,1.216819e+06,2.205304e+07,5.706478e+07,1.144776e+07,1.606177e+07,2.060164e+07,4.134599e+06,...,3.199018e+09,0.0,0.0,0.0,0.0,0.0,0.0,6.557035e+08,3.854722e+09,0.0
